In [1]:
import pandas as pd
import gensim
from gensim import models
import sys
import glob
import os
import pickle
from gensim.similarities.docsim import SparseMatrixSimilarity

pd.options.display.max_rows = 10

##### --- Download stage skipped ---

##### --- Cleaning stage skipped ---

##### --- Tokenizing stage skipped ---

##### --- Grouping stage skipped ---

In [2]:
# As for demo usage, only 100 programmes are included.

documents = pickle.load(open("data/tokensdf.pickle.df", 'rb'))
documents

programme_id
5614401466    [阿柔, 阿柔, 之前, 是, 迫不得已, 才, 揭, 你, 陰私, 我, 不是, 想, 傷...
4066115555    [早安, 你, 繼續, 睡, 吧, 你, 沒, 必要, 特地, 起床, 幫, 我, 釘, 畫...
1753567166    [尋龍記, 一億年, 角龍, 的, 世界, 即, 將展, 開, 一億, 年前, 在, 這片,...
3005551154    [朗, 翹, 我少, 說, 也, 做, 了, 全職, 主婦, 有, 七年, 了, 不過, 我...
3561536243    [月桂, 樹西, 裝店, 第, 10, 集, 爸爸, 好久, 不見, 了, 是, 我, 錯,...
                                    ...                        
2110232460    [二師兄, 師嫂, 看見, 你們, 真, 好, 你, 為, 何在, 這兒, 師父, 叫, 我...
2446564632    [詩經, 說, 到, 八月, 剝棗, 十月, 獲稻, 在, 稻熟, 之前, 以, 稻田, 落...
2173235267    [想念, 你, 我, 再也, 不會, 假裝, 不, 認識, 你, 了, 知道, 嗎, 我, ...
2133252602    [美食, 任務前, 進桃園, 帶, 你, 發掘, 三種, 好, 味道, 異國, 故鄉, 好,...
2070600742    [仁哥, 吃, 什麼, 早餐, Ｄ, 熱, 咖啡, 謝謝, 好, 知道, 最, 後, 一份,...
Name: tokens, Length: 100, dtype: object

### Create the dictionary of the corpus

In [3]:
dictionary = gensim.corpora.Dictionary(documents)

In [4]:
dictionary.filter_extremes(no_above=0.5)

In [5]:
# dictionary.save('gensim.dictionary')

### Create bag-of-word formatted corpus

In [6]:
bow_corpus = [dictionary.doc2bow(doc) for doc in documents]

### Go thru TF-IDF layer

In [7]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [8]:
# tfidf.save('tfidf.model')

### Train the LDA model

In [10]:
lda_model = gensim.models.LdaMulticore(
    corpus_tfidf, 
    num_topics=100,      # Increase the number of topics in real case.
    id2word=dictionary, 
    passes=2,            # Increase the number of passes in real case.
    iterations=200,      # Increase the number of iterations in real case.
    workers=5,
)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1023: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


### Evaluate result

In [11]:
corpus_in_LDA_space = lda_model[tfidf[bow_corpus]]

In [12]:
sim_index = SparseMatrixSimilarity(corpus_in_LDA_space, num_features=lda_model.num_topics)

In [13]:
my_index = 49

print("Similar item for id:%s" % documents.index[my_index])
print("==============================")
scores = sim_index[corpus_in_LDA_space[my_index]]
for position in (-scores).argsort()[:10]:
    if position != my_index:
        print("id:%s (Score: %.5f)" % (documents.index[position], scores[position]))


Similar item for id:3725633637
id:3561536243 (Score: 0.45741)
id:1536741054 (Score: 0.38966)
id:2070600742 (Score: 0.38789)
id:1726651554 (Score: 0.32497)
id:3062776333 (Score: 0.31964)
id:3516060230 (Score: 0.31391)
id:1424765452 (Score: 0.30046)
id:5614401466 (Score: 0.29848)
id:3005551154 (Score: 0.29343)
